In [4]:
import torch
import torchvision
import torchvision.transforms as transforms


class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 3 32 32 -> 32 16 16
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        # 32 16 16 -> 64 8 8
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc = torch.nn.Linear(8 * 8 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)  # 가중치 초기화 방식

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)  # flatten
        out = self.fc(out)
        return out


def train_model(device, train_loader, validation_loader, learning_rate, training_epochs, model_name):
    # 모델 및 학습방법 정의
    model = CNN().to(device)
    criterion = torch.nn.CrossEntropyLoss().to(device)  # softmax 포함되어 있음
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    total_batch = len(train_loader)
    validation_len = len(validation_loader)
    for epoch in range(training_epochs):
        # 학습
        avg_cost = 0
        for X, Y in train_loader:  # 미니 배치 단위로 꺼내온다, X = 미니배치, Y = label
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()
            hypothesis = model(X)
            cost = criterion(hypothesis, Y)
            cost.backward()
            optimizer.step()

            avg_cost += cost / total_batch
        # validation data 검증
        with torch.no_grad():
            avg_accuracy = 0
            for x, y in validation_loader:
                x_test = x.to(device)
                y_test = y.to(device)
                prediction = model(x_test)
                correct_prediction = torch.argmax(prediction, 1) == y_test
                avg_accuracy += correct_prediction.float().mean()
            avg_accuracy = avg_accuracy / validation_len
            print(f"[Epoch: {epoch+1:>4}] cost = {avg_cost:>.9} validation_accuracy = {avg_accuracy:>.9}")

    torch.save(model.state_dict(), f"./data/{model_name}.pt")


def run_model(device, test_loader, model_name):
    model = CNN().to(device)
    model.load_state_dict(torch.load(f"./data/{model_name}.pt"))
    model.eval()
    test_len = len(test_loader)
    with torch.no_grad():
        avg_accuracy = 0
        for x, y in test_loader:
            x_test = x.to(device)
            y_test = y.to(device)
            prediction = model(x_test)
            correct_prediction = torch.argmax(prediction, 1) == y_test
            avg_accuracy += correct_prediction.float().mean()
        avg_accuracy = avg_accuracy / test_len
        print(f"test_accuracy = {avg_accuracy:>.9}")


def get_data(batch_size, num_workers):
    # train, validation, test dataset
    # classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

    # transform.Compose: data를 원하는 형태로 가공
    # normalize: (value-mean)/std, 3ch(rgb)라 3개, 0~1을 -1~1로 바꿈, transforms.Normalize((mean1, mean2, mean3), (std1, std2, std3)),
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    # train and validation set
    tmp_set = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    train_len = int(len(tmp_set)*0.9)
    val_len = len(tmp_set)-train_len
    train_set, validation_set = torch.utils.data.random_split(tmp_set,[train_len,val_len])
    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True
    )
    validation_loader = train_loader = torch.utils.data.DataLoader(
        validation_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True
    )

    # test set
    test_set = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
    test_loader = train_loader = torch.utils.data.DataLoader(
        test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True
    )

    return train_loader, validation_loader, test_loader


def main():
    # device setup
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.manual_seed(777)
    if device == "cuda":
        print("cuda avaliable")
        torch.cuda.manual_seed_all(777)

    # hyper parameter
    num_workers = 4  # cpu 사용량, 스레드의 절반 정도면 무난하다.
    batch_size = 256
    learning_rate = 0.001
    training_epochs = 30
    model_name = "cnn_256_001_30"

    # run 
    print(model_name)
    train_loader, validation_loader, test_loader = get_data(batch_size, num_workers)
    train_model(device, train_loader, validation_loader, learning_rate, training_epochs, model_name)
    run_model(device, test_loader, model_name)


if __name__ == "__main__":
    # 멀티태스킹 오류 안나려면 이렇게 실행해야 한다
    main()


cuda avaliable
cnn_256_001_30
Files already downloaded and verified
Files already downloaded and verified
[Epoch:    1] cost = 1.93356335 validation_accuracy = 0.401521385
[Epoch:    2] cost = 1.53720927 validation_accuracy = 0.474712163
[Epoch:    3] cost = 1.36135423 validation_accuracy = 0.513774693
[Epoch:    4] cost = 1.25458431 validation_accuracy = 0.536800981
[Epoch:    5] cost = 1.18094087 validation_accuracy = 0.545230269
[Epoch:    6] cost = 1.11965942 validation_accuracy = 0.554893076
[Epoch:    7] cost = 1.06377602 validation_accuracy = 0.565583885
[Epoch:    8] cost = 1.01191759 validation_accuracy = 0.581620097
[Epoch:    9] cost = 0.9640131 validation_accuracy = 0.589432597
[Epoch:   10] cost = 0.918674529 validation_accuracy = 0.595600307
[Epoch:   11] cost = 0.876574397 validation_accuracy = 0.600328922
[Epoch:   12] cost = 0.835867047 validation_accuracy = 0.603207231
[Epoch:   13] cost = 0.796459794 validation_accuracy = 0.607524693
[Epoch:   14] cost = 0.75885725 v

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms


class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 3 32 32 -> 32 16 16
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        # 32 16 16 -> 64 8 8
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc = torch.nn.Linear(8 * 8 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)  # 가중치 초기화 방식

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)  # flatten
        out = self.fc(out)
        return out


def train_model(device, train_loader, validation_loader, learning_rate, training_epochs, model_name):
    # 모델 및 학습방법 정의
    model = CNN().to(device)
    criterion = torch.nn.CrossEntropyLoss().to(device)  # softmax 포함되어 있음
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    total_batch = len(train_loader)
    validation_len = len(validation_loader)
    for epoch in range(training_epochs):
        # 학습
        avg_cost = 0
        for X, Y in train_loader:  # 미니 배치 단위로 꺼내온다, X = 미니배치, Y = label
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()
            hypothesis = model(X)
            cost = criterion(hypothesis, Y)
            cost.backward()
            optimizer.step()

            avg_cost += cost / total_batch
        # validation data 검증
        with torch.no_grad():
            avg_accuracy = 0
            for x, y in validation_loader:
                x_test = x.to(device)
                y_test = y.to(device)
                prediction = model(x_test)
                correct_prediction = torch.argmax(prediction, 1) == y_test
                avg_accuracy += correct_prediction.float().mean()
            avg_accuracy = avg_accuracy / validation_len
            print(f"[Epoch: {epoch+1:>4}] cost = {avg_cost:>.9} validation_accuracy = {avg_accuracy:>.9}")

    torch.save(model.state_dict(), f"./data/{model_name}.pt")


def run_model(device, test_loader, model_name):
    model = CNN().to(device)
    model.load_state_dict(torch.load(f"./data/{model_name}.pt"))
    model.eval()
    test_len = len(test_loader)
    with torch.no_grad():
        avg_accuracy = 0
        for x, y in test_loader:
            x_test = x.to(device)
            y_test = y.to(device)
            prediction = model(x_test)
            correct_prediction = torch.argmax(prediction, 1) == y_test
            avg_accuracy += correct_prediction.float().mean()
        avg_accuracy = avg_accuracy / test_len
        print(f"test_accuracy = {avg_accuracy:>.9}")


def get_data(batch_size, num_workers):
    # train, validation, test dataset
    # classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

    # transform.Compose: data를 원하는 형태로 가공
    # normalize: (value-mean)/std, 3ch(rgb)라 3개, 0~1을 -1~1로 바꿈, transforms.Normalize((mean1, mean2, mean3), (std1, std2, std3)),
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    # train and validation set
    tmp_set = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    train_len = int(len(tmp_set)*0.9)
    val_len = len(tmp_set)-train_len
    train_set, validation_set = torch.utils.data.random_split(tmp_set,[train_len,val_len])
    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True
    )
    validation_loader = train_loader = torch.utils.data.DataLoader(
        validation_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True
    )

    # test set
    test_set = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
    test_loader = train_loader = torch.utils.data.DataLoader(
        test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True
    )

    return train_loader, validation_loader, test_loader


def main():
    # device setup
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.manual_seed(777)
    if device == "cuda":
        print("cuda avaliable")
        torch.cuda.manual_seed_all(777)

    # hyper parameter
    num_workers = 4  # cpu 사용량, 스레드의 절반 정도면 무난하다.
    batch_size = 16
    learning_rate = 0.0001
    training_epochs = 30
    model_name = "cnn_16_0001_30"

    # run 
    print(model_name)
    train_loader, validation_loader, test_loader = get_data(batch_size, num_workers)
    train_model(device, train_loader, validation_loader, learning_rate, training_epochs, model_name)
    run_model(device, test_loader, model_name)


if __name__ == "__main__":
    # 멀티태스킹 오류 안나려면 이렇게 실행해야 한다
    main()


cuda avaliable
cnn_16_0001_30
Files already downloaded and verified
Files already downloaded and verified
[Epoch:    1] cost = 1.94043493 validation_accuracy = 0.399839759
[Epoch:    2] cost = 1.62733495 validation_accuracy = 0.452323735
[Epoch:    3] cost = 1.48007774 validation_accuracy = 0.480368584
[Epoch:    4] cost = 1.39336526 validation_accuracy = 0.502203524
[Epoch:    5] cost = 1.32964492 validation_accuracy = 0.517828524
[Epoch:    6] cost = 1.27959692 validation_accuracy = 0.530248404
[Epoch:    7] cost = 1.23857164 validation_accuracy = 0.535056114
[Epoch:    8] cost = 1.20333135 validation_accuracy = 0.541065693
[Epoch:    9] cost = 1.17212951 validation_accuracy = 0.547475994
[Epoch:   10] cost = 1.14355958 validation_accuracy = 0.555088162
[Epoch:   11] cost = 1.11691427 validation_accuracy = 0.559094548
[Epoch:   12] cost = 1.09162939 validation_accuracy = 0.562900662
[Epoch:   13] cost = 1.06753898 validation_accuracy = 0.568309307
[Epoch:   14] cost = 1.04439366 vali

In [8]:
import torch
import torchvision
import torchvision.transforms as transforms


class CNN2(torch.nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        # 3 32 32 -> 32 16 16
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        # 32 16 16 -> 64 8 8
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        # 64 8 8 -> 128 4 4
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc = torch.nn.Linear(4 * 4 * 128, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)  # 가중치 초기화 방식

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)  # flatten
        out = self.fc(out)
        return out


def train_model(device, train_loader, validation_loader, learning_rate, training_epochs, model_name):
    # 모델 및 학습방법 정의
    model = CNN2().to(device)
    criterion = torch.nn.CrossEntropyLoss().to(device)  # softmax 포함되어 있음
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    total_batch = len(train_loader)
    validation_len = len(validation_loader)
    for epoch in range(training_epochs):
        # 학습
        avg_cost = 0
        for X, Y in train_loader:  # 미니 배치 단위로 꺼내온다, X = 미니배치, Y = label
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()
            hypothesis = model(X)
            cost = criterion(hypothesis, Y)
            cost.backward()
            optimizer.step()

            avg_cost += cost / total_batch
        # validation data 검증
        with torch.no_grad():
            avg_accuracy = 0
            for x, y in validation_loader:
                x_test = x.to(device)
                y_test = y.to(device)
                prediction = model(x_test)
                correct_prediction = torch.argmax(prediction, 1) == y_test
                avg_accuracy += correct_prediction.float().mean()
            avg_accuracy = avg_accuracy / validation_len
            print(f"[Epoch: {epoch+1:>4}] cost = {avg_cost:>.9} validation_accuracy = {avg_accuracy:>.9}")

    torch.save(model.state_dict(), f"./data/{model_name}.pt")


def run_model(device, test_loader, model_name):
    model = CNN2().to(device)
    model.load_state_dict(torch.load(f"./data/{model_name}.pt"))
    model.eval()
    test_len = len(test_loader)
    with torch.no_grad():
        avg_accuracy = 0
        for x, y in test_loader:
            x_test = x.to(device)
            y_test = y.to(device)
            prediction = model(x_test)
            correct_prediction = torch.argmax(prediction, 1) == y_test
            avg_accuracy += correct_prediction.float().mean()
        avg_accuracy = avg_accuracy / test_len
        print(f"test_accuracy = {avg_accuracy:>.9}")


def get_data(batch_size, num_workers):
    # train, validation, test dataset
    # classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

    # transform.Compose: data를 원하는 형태로 가공
    # normalize: (value-mean)/std, 3ch(rgb)라 3개, 0~1을 -1~1로 바꿈, transforms.Normalize((mean1, mean2, mean3), (std1, std2, std3)),
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    # train and validation set
    tmp_set = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    train_len = int(len(tmp_set)*0.9)
    val_len = len(tmp_set)-train_len
    train_set, validation_set = torch.utils.data.random_split(tmp_set,[train_len,val_len])
    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True
    )
    validation_loader = train_loader = torch.utils.data.DataLoader(
        validation_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True
    )

    # test set
    test_set = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
    test_loader = train_loader = torch.utils.data.DataLoader(
        test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True
    )

    return train_loader, validation_loader, test_loader


def main():
    # device setup
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.manual_seed(777)
    if device == "cuda":
        print("cuda avaliable")
        torch.cuda.manual_seed_all(777)

    # hyper parameter
    num_workers = 4  # cpu 사용량, 스레드의 절반 정도면 무난하다.
    batch_size = 256
    learning_rate = 0.001
    training_epochs =30
    model_name = "cnn2_256_001_30"

    # run 
    print(model_name)
    train_loader, validation_loader, test_loader = get_data(batch_size, num_workers)
    train_model(device, train_loader, validation_loader, learning_rate, training_epochs, model_name)
    run_model(device, test_loader, model_name)


if __name__ == "__main__":
    # 멀티태스킹 오류 안나려면 이렇게 실행해야 한다
    main()


cuda avaliable
cnn2_256_001_30
Files already downloaded and verified
Files already downloaded and verified
[Epoch:    1] cost = 1.9608258 validation_accuracy = 0.387335539
[Epoch:    2] cost = 1.56321871 validation_accuracy = 0.477796048
[Epoch:    3] cost = 1.39663792 validation_accuracy = 0.497532904
[Epoch:    4] cost = 1.29948747 validation_accuracy = 0.502672672
[Epoch:    5] cost = 1.21195579 validation_accuracy = 0.530016422
[Epoch:    6] cost = 1.1338861 validation_accuracy = 0.549547672
[Epoch:    7] cost = 1.06580412 validation_accuracy = 0.5625
[Epoch:    8] cost = 1.00681794 validation_accuracy = 0.576685846
[Epoch:    9] cost = 0.959876776 validation_accuracy = 0.601151347
[Epoch:   10] cost = 0.909274697 validation_accuracy = 0.615748346
[Epoch:   11] cost = 0.848771036 validation_accuracy = 0.631990135
[Epoch:   12] cost = 0.79173708 validation_accuracy = 0.632606924
[Epoch:   13] cost = 0.738178134 validation_accuracy = 0.636101961
[Epoch:   14] cost = 0.687303424 valid